In [ ]:
# ================================
# 📊 Analysis: Average Age by Quartile and Role (Monthly and Overall)
# ================================

import pandas as pd  # Importing the pandas library for data manipulation and analysis.
import locale  # Importing locale to handle localization settings.

# =========================
# 0. Regional Configuration (for months in PT-BR)
# =========================
try:
    # Set the locale for Linux/macOS systems to Brazilian Portuguese for date formatting.
    locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')
except:
    try:
        # Set the locale for Windows systems to Brazilian Portuguese.
        locale.setlocale(locale.LC_TIME, 'Portuguese_Brazil')
    except:
        # If it fails to set the locale, print a warning message.
        pass  # Handle any exceptions that may occur.

# =========================
# 1. Load the dataset
# =========================
file_path = r'C:\\Users\\FabianaKuhlmann\\Downloads\\Cruzamento quartil x idade.xlsx'  # File path for the input Excel file.
df = pd.read_excel(file_path, sheet_name='Base')  # Load the data from the specified sheet into a pandas DataFrame.

# =========================
# 2. Standardize column names
# =========================
df.columns = df.columns.str.strip().str.lower()  # Remove whitespace from column names and convert them to lowercase.

# Example of expected columns (after converting to lowercase):
# ['collaborator', 'function', 'overall_quartile_position', 'age', 'period (adjusted)']

# =========================
# 3. Create year and month columns (if 'period (adjusted)' exists)
# =========================
if 'período (ajustado)' in df.columns:  # Check if the 'adjusted period' column exists.
    df['período (ajustado)'] = pd.to_datetime(df['período (ajustado)'], errors='coerce')  # Convert the column to datetime, coerce errors.
    df['ano'] = df['período (ajustado)'].dt.year  # Extract the year from the date.
    df['mes'] = df['período (ajustado)'].dt.strftime('%b')  # Format the month as a three-letter abbreviation (e.g., 'jul', 'ago').

# =========================
# 4. Basic cleaning
# =========================
required_columns = ['função', 'posicaoquartil_geral', 'idade']  # List of required columns for analysis.
missing = [c for c in required_columns if c not in df.columns]  # Check for missing required columns.
if missing:  # If there are missing columns, raise an error.
    raise ValueError(f"The following required columns are missing: {missing}")

# Drop rows with missing values in the specified critical columns.
df = df.dropna(subset=['função', 'posicaoquartil_geral', 'idade'])
df['idade'] = pd.to_numeric(df['idade'], errors='coerce')  # Convert the 'age' column to numeric, coercing errors.

# =========================
# 5. 📅 Monthly Analysis
# =========================
average_age_monthly = (
    df.groupby(['ano', 'mes', 'função', 'posicaoquartil_geral'], as_index=False)  # Group by year, month, function, and quartile position.
      .agg(average_age=('idade', 'mean'),  # Calculate the average age.
           number_of_employees=('idade', 'count'))  # Count the number of employees.
)

# Define the ordered list of months.
ordered_months = ['jan', 'fev', 'mar', 'abr', 'mai', 'jun', 'jul', 'ago', 'set', 'out', 'nov', 'dez']
average_age_monthly['mes'] = pd.Categorical(  # Set the month column as a categorical variable with defined order.
    average_age_monthly['mes'].str.lower(),
    categories=ordered_months,
    ordered=True
)
average_age_monthly = average_age_monthly.sort_values(['ano', 'mes', 'função', 'posicaoquartil_geral'])  # Sort the DataFrame.

# =========================
# 6. 🧮 Overall Analysis (not considering the month)
# =========================
average_age_overall = (
    df.groupby(['função', 'posicaoquartil_geral'], as_index=False)  # Group by function and quartile position.
      .agg(average_age=('idade', 'mean'),  # Calculate average age.
           number_of_employees=('idade', 'count'))  # Count the number of employees.
      .sort_values(['função', 'posicaoquartil_geral'])  # Sort the DataFrame.
)

# =========================
# 7. Export Results (two sheets in the same file)
# =========================
output_path = r'C:\\Users\\FabianaKuhlmann\\Downloads\\media_idade_por_quartil.xlsx'  # File path for the output Excel file.

# Writing the results to an Excel file with two sheets.
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    average_age_monthly.to_excel(writer, index=False, sheet_name='Monthly')  # Save monthly average age data.
    average_age_overall.to_excel(writer, index=False, sheet_name='Overall')  # Save overall average age data.

# =========================
# 8. Display Summary in the Terminal
# =========================
print("✅ Analyses completed successfully!")  # Confirmation message.
print("File saved at:", output_path)  # Output file location.
print("\n📅 Preview of monthly analysis:")  # Header for monthly analysis preview.
print(average_age_monthly.head(10))  # Display the first 10 rows of monthly analysis.
print("\n🧮 Preview of overall analysis:")  # Header for overall analysis preview.
print(average_age_overall.head(10))  # Display the first 10 rows of overall analysis.
